In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pysal.lib import weights
from pysal.model import spreg
from scipy.stats.mstats import zscore
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tools.tools import add_constant
np.random.seed(0)

shp_path = 'data/tracts_shapefile/tracts_shapefile.shp'
indicators_path = 'data/tracts_indicators_grades_eras_index.csv'
response = 'grid_index_geom'

spat_diag = False

In [2]:
df = pd.read_csv(indicators_path, dtype={'geoid':str, 'state':str, 'county':str})
df.shape

(72663, 154)

In [3]:
gdf = gpd.read_file(shp_path).set_index('GEOID')
gdf.shape

(74133, 12)

In [4]:
# restrict modeling to only urban tracts
df = df[df['is_urban'] == 1]
df.shape

(46311, 154)

In [5]:
df.corr()[response].abs().sort_values(ascending=False).head(10)

grid_index_geom        1.000000
grid_index             0.955074
prop_4way              0.903810
orientation_entropy    0.869145
orientation_order      0.867383
length_entropy_log     0.709035
prop_deadend           0.690659
straightness           0.659883
circuity_avg           0.604262
k_avg                  0.546986
Name: grid_index_geom, dtype: float64

## Modeling

In [6]:
df['pop_density'] = df['pop_density'] / 1000 #put these regression coefficients into units of 1000s of persons

In [7]:
# identify the era dummies in the dataframe
era_primary_dummies = sorted([c for c in df.columns if 'dummy_primary_' in c and '_1939_earlier' not in c])

# get the state dummies
states = df['state_abbrev'].unique()
state_dummies = states[1:].tolist()
#state_dummies = sorted([s for s in states if s != 'CA']) #all but CA
len(state_dummies)

50

In [8]:
%%time
# create county dummies
df['st_county'] = df['state'].astype(str) + df['county'].astype(str)
counties = df['st_county'].unique()
for county in counties:
    df[county] = df['st_county'].map(lambda x: 1 if x==county else 0)

county_dummies = counties[1:].tolist()
#county_dummies = sorted([c for c in counties if c != '06037']) #all but LA county
print(len(county_dummies))

1390
Wall time: 24 s


In [9]:
# define which dummies to use as the spatial fixed effects
# if including both county + state, you'll get colinearity unless you drop one county from each state?
fixed_effects = county_dummies #+ state_dummies
len(fixed_effects)

1390

In [10]:
def get_response_and_design(df, response, predictors, condition_number=True):
    
    # select predictors and drop any rows with nulls in the response or predictors
    df_model = df.replace([np.inf, -np.inf], np.nan).dropna(subset=predictors + [response])

    # create design matrix and response vector (and response as matrix for pysal)
    X = df_model[predictors]
    y = df_model[response]

    # drop columns that are constants (to prevent perfect colinearity)
    # this happens if a county has no observations, for instance
    X = X.loc[:, X.nunique() != 1]
    
    # what are the geoids of the observations retained in the response vector + design matrix?
    geoids = df_model['geoid'].values
    
    if condition_number:
        cn = np.linalg.cond(zscore(X))
        return y, X, geoids, cn
    else:
        return y, X, geoids

In [11]:
def make_pysal_table(model, precision=4, ignore=None):
    
    try:
        idx = model.name_z
    except:
        idx = model.name_x
    
    z_stat = np.array(model.z_stat)
    table = pd.DataFrame({'beta' : model.betas.flatten(),
                          's.e.' : model.std_err,
                          'z'    : z_stat[:, 0],
                          'p'    : z_stat[:, 1]}, 
                          index=idx)
    
    if ignore is not None:
        to_drop = [c for c in ignore if c in table.index]
        table = table.drop(to_drop, axis='rows')
    
    return table.round(precision)

## Model 0

check the grid index vs its components to check its validity

In [12]:
%%time
regressors0 = ['orientation_order', 'prop_4way', 'straightness']
predictors0 = regressors0
y, X, geoids, cn = get_response_and_design(df, response, predictors0)
print(cn)

2.4156406511054223
Wall time: 17.9 s


In [13]:
%%time
# estimate the model with OLS
result0 = sm.OLS(y, add_constant(X)).fit()
print(result0.rsquared)

G:\Anaconda\envs\ox\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


0.9722518203503049
Wall time: 103 ms


## Model 1a: OLS

predict grid index from era dummies, spatial fixed effects, and controls

In [14]:
%%time
regressors1 = ['aland', 'pop_density', 'prop_single_fam', 'med_rooms_per_home', #settlement density/scale
               'intersect_density', 'length_mean', #street spatial scale
               'elevations_iqr', 'grade_mean'] #hilliness
predictors1 = era_primary_dummies + regressors1 + fixed_effects
y, X, geoids, cn = get_response_and_design(df, response, predictors1)
print(cn)

186.08838946034666
Wall time: 27.5 s


In [15]:
%%time
# estimate the model with OLS
result1 = sm.OLS(y, add_constant(X)).fit()
print(result1.rsquared)

0.675528802926695
Wall time: 14.9 s


In [16]:
regressors = pd.Series(['const'] + era_primary_dummies + regressors1).drop_duplicates(keep='first').tolist()
print(summary_col(results=result1, regressor_order=regressors, drop_omitted=True, stars=True))


                              grid_index_geom
---------------------------------------------
const                         0.6773***      
                              (0.0526)       
dummy_primary_prop_1940_49    -0.0396***     
                              (0.0036)       
dummy_primary_prop_1950_59    -0.0851***     
                              (0.0019)       
dummy_primary_prop_1960_69    -0.1413***     
                              (0.0023)       
dummy_primary_prop_1970_79    -0.1734***     
                              (0.0021)       
dummy_primary_prop_1980_89    -0.1975***     
                              (0.0025)       
dummy_primary_prop_1990_99    -0.1917***     
                              (0.0027)       
dummy_primary_prop_2000_09    -0.1567***     
                              (0.0028)       
dummy_primary_prop_2010_later -0.1173***     
                              (0.0122)       
aland                         -6.1799***     
                              (0.

## Model 1b: spatially explicit version

In [17]:
%%time
# calculate spatial weights matrix for spatially-explicit alternative specification
W1 = weights.Queen.from_dataframe(gdf.loc[geoids], silence_warnings=True)
W1.transform = 'r'

Wall time: 44.5 s


In [18]:
%%time
# check ols diagnostics to see nature of spatial dependence
Y = pd.DataFrame(y)
if spat_diag:
    ols = spreg.ols.OLS(y=Y.values, x=X.values, w=W1, spat_diag=True, moran=True)
    print(ols.moran_res)
    print(ols.rlm_lag, ols.rlm_error)

Wall time: 1 ms


In [19]:
%%time
# spatial lag model uses w*Y as endogenous var
wY = weights.lag_spatial(W1, Y)

# use w*X and w*w*X as instruments for 2SLS
# do not include spatial fixed effects (as w*X wouldn't make sense with them in it)
wX = weights.lag_spatial(W1, X[regressors1 + era_primary_dummies])
wwX = weights.lag_spatial(W1, wX)
q = np.concatenate([wX, wwX], axis=1)

Wall time: 222 ms


In [20]:
%%time
# spatial lag model via TSLS
# can't use GM_Lag here, because it doesn't let you use a reduced set of X's columns as instruments (to avoid including spatial fixed effects)
tsls_model1 = spreg.twosls.TSLS(y=Y.values, x=X.values, w=W1, name_w='W1', yend=wY, q=q, robust='white', spat_diag=True,
                                name_x=X.columns.tolist(), name_y=response)

table1 = make_pysal_table(tsls_model1, ignore=fixed_effects)
print(tsls_model1.pr2)
print(tsls_model1.n)
print(table1)

0.736819257317994
46157
                                 beta    s.e.        z       p
CONSTANT                       0.4669  0.0407  11.4637  0.0000
dummy_primary_prop_1940_49    -0.0358  0.0036  -9.8195  0.0000
dummy_primary_prop_1950_59    -0.0690  0.0021 -33.0625  0.0000
dummy_primary_prop_1960_69    -0.1105  0.0025 -43.7145  0.0000
dummy_primary_prop_1970_79    -0.1325  0.0025 -52.7957  0.0000
dummy_primary_prop_1980_89    -0.1497  0.0028 -53.1600  0.0000
dummy_primary_prop_1990_99    -0.1456  0.0031 -47.3504  0.0000
dummy_primary_prop_2000_09    -0.1183  0.0033 -35.9190  0.0000
dummy_primary_prop_2010_later -0.0939  0.0115  -8.1928  0.0000
aland                         -5.2862  0.2542 -20.7960  0.0000
pop_density                    0.0037  0.0003  13.4201  0.0000
prop_single_fam                0.0477  0.0036  13.3949  0.0000
med_rooms_per_home            -0.0227  0.0008 -28.1345  0.0000
intersect_density              0.0009  0.0000  20.9041  0.0000
length_mean                    

In [21]:
# if significant, spatial error autocorrelation is present
tsls_model1.ak_test

# you might want a combo model if ak stat is significant
# and if the combo model parameter estimates differ greatly from the TSLS spatial lag model
# you can't run GM_Combo_Het with spatial fixed effects 
# because it doesn't let you use a reduced set of X's columns as instruments (to avoid including spatial fixed effects)
# instead, mimic the TSLS approach above, via GM_Endog_Error_Het
# (but if you run the line below, you see similar parameter estimates, so this is unnecessary)
#combo_model = spreg.GM_Endog_Error_Het(y=Y.values, x=X.values, w=W1, name_w='W1', yend=wY, q=q,
#                                       name_x=X.columns.tolist(), name_y=response)

(358.8559480631558, 4.9968816551316184e-80)

In [22]:
table1.to_csv('data/table1.csv', index=True)

#### Interpret total, direct, and indirect effects of spatial lag model

Spatial lag model coefficients are not purely marginal effects:

  - we're also interested in the total effect of a unit change in predictor k on the response
  - total effect = direct effect (k's estimated coefficient) + indirect effect (spatial spillover)
  - total effect is the change in response if you make a unit change in k at all locations simultaneously
  - direct effect is what happens locally if you make a change (in k) at that one location
  - indirect effect is the local effect of spillover from making that change at all other locations

In [23]:
def impacts(variable, model):
    idx = model.name_x.index(variable)
    direct_effect = model.betas[idx][0]
    rho = tsls_model1.betas[-1, 0]
    total_effect = direct_effect / (1 - rho)
    indirect_effect = total_effect - direct_effect
    return total_effect, direct_effect, indirect_effect

effects = {}
for variable in [c for c in tsls_model1.name_x if c not in fixed_effects]:
    effects[variable] = impacts(variable, tsls_model1)
pd.DataFrame(effects, index=['TE', 'DE', 'IE']).T

,TE,DE,IE
CONSTANT,0.699491,0.466919,0.232573
dummy_primary_prop_1940_49,-0.053654,-0.035815,-0.017839
dummy_primary_prop_1950_59,-0.103360,-0.068994,-0.034366
dummy_primary_prop_1960_69,-0.165587,-0.110532,-0.055056
dummy_primary_prop_1970_79,-0.198544,-0.132530,-0.066013
dummy_primary_prop_1980_89,-0.224276,-0.149707,-0.074569
dummy_primary_prop_1990_99,-0.218107,-0.145589,-0.072518
dummy_primary_prop_2000_09,-0.177264,-0.118326,-0.058938
dummy_primary_prop_2010_later,-0.140672,-0.093900,-0.046772
aland,-7.919279,-5.286212,-2.633067
